<a href="https://colab.research.google.com/github/MiniRedTrout/Segmentation/blob/main/AortaSegmentation_2d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импорт библиотек

In [87]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import os
import scipy.io
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import shutil

Путь датасета с kaggle

In [88]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("miniredtrout/abdominalaorta")

Using Colab cache for faster access to the 'abdominalaorta' dataset.


In [89]:
path

'/kaggle/input/abdominalaorta'

In [90]:
!pip install hydra-core omegaconf
import hydra
from omegaconf import DictConfig, OmegaConf
import os

In [91]:
!pip install clearml

In [92]:
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=XGMS61QRIXH7G4URU0X7VGTDAIQUVE
%env CLEARML_API_SECRET_KEY=3mpOB_Br37BLBntBoUvtxmfbBfmkbkdfRUYGje8GfL1gwe_ufJd1w4UzuQWOPaAtTWE

env: CLEARML_WEB_HOST=https://app.clear.ml/
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=XGMS61QRIXH7G4URU0X7VGTDAIQUVE
env: CLEARML_API_SECRET_KEY=3mpOB_Br37BLBntBoUvtxmfbBfmkbkdfRUYGje8GfL1gwe_ufJd1w4UzuQWOPaAtTWE


In [93]:
from clearml import Task, Logger

In [94]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
import hydra
from omegaconf import DictConfig, OmegaConf

config_dir = '/content/drive/MyDrive/segmentation_project4/config'
import shutil
if os.path.exists(config_dir):
    shutil.rmtree(config_dir)

os.makedirs(config_dir, exist_ok=True)

simple_config = """
training:
  batch_size: 16
  epochs: 40
  learning_rate: 0.0002
  optimizer: "AdamW"
  weight_decay: 0.01

data:
  original_images_dir: "/kaggle/input/abdominalaorta/images"
  original_labels_dir: "/kaggle/input/abdominalaorta/labels"
  output_base_dir: "/content/base_patients"
  test_size: 0.2
  random_state: 42
  image_size: 256
  num_workers: 2

model:
  in_channel: 1
  out_channel: 2

transforms:
  image_size: 256
  augmentation:
    horizontal_flip_p: 0.5
    vertical_flip_p: 0.3
    rotate90_p: 0.5
    affine_p: 0.5
    brightness_contrast_p: 0.3
    noise_p: 0.2
"""

with open(f'{config_dir}/config.yaml', 'w') as f:
    f.write(simple_config)

def load_hydra_config():
    config_path = "/content/drive/MyDrive/segmentation_project/config"
    config_name = "config"

    with hydra.initialize_config_dir(config_dir=config_path, version_base="1.2"):
        cfg = hydra.compose(config_name=config_name)

    OmegaConf.set_struct(cfg, False)

    return cfg

Mounted at /content/drive


Разделение на train test выборку по пациентам

In [95]:
def create_train_test_split_by_patient():
    cfg = load_hydra_config()
    patient_folders = []
    for folder in os.listdir(cfg.data.original_images_dir):
        folder_path = os.path.join(cfg.data.original_images_dir, folder)
        if (os.path.isdir(folder_path) and
            os.path.exists(os.path.join(cfg.data.original_labels_dir, folder))):
            patient_folders.append(folder)

    train_patients, test_patients = train_test_split(
        patient_folders,
        test_size=cfg.data.test_size,
        random_state=cfg.data.random_state
    )

    train_images_dir = os.path.join(cfg.data.output_base_dir, 'train', 'images')
    train_labels_dir = os.path.join(cfg.data.output_base_dir, 'train', 'labels')
    test_images_dir = os.path.join(cfg.data.output_base_dir, 'test', 'images')
    test_labels_dir = os.path.join(cfg.data.output_base_dir, 'test', 'labels')

    for dir_path in [train_images_dir, train_labels_dir, test_images_dir, test_labels_dir]:
        os.makedirs(dir_path, exist_ok=True)

    def copy_patient_files(patients, src_images_dir, src_labels_dir, dst_images_dir, dst_labels_dir):
        total_files = 0
        for patient in patients:
            patient_images_dir = os.path.join(src_images_dir, patient)
            patient_labels_dir = os.path.join(src_labels_dir, patient)

            for file in os.listdir(patient_images_dir):
                if file.endswith('.png'):
                    src_image_path = os.path.join(patient_images_dir, file)
                    dst_image_path = os.path.join(dst_images_dir, f"{patient}_{file}")
                    shutil.copy2(src_image_path, dst_image_path)

                    src_label_path = os.path.join(patient_labels_dir, file)
                    dst_label_path = os.path.join(dst_labels_dir, f"{patient}_{file}")
                    shutil.copy2(src_label_path, dst_label_path)

                    total_files += 1

        return total_files

    train_total = copy_patient_files(train_patients, cfg.data.original_images_dir, cfg.data.original_labels_dir,
                                   train_images_dir, train_labels_dir)

    test_total = copy_patient_files(test_patients, cfg.data.original_images_dir, cfg.data.original_labels_dir,
                                  test_images_dir, test_labels_dir)

    return {
        'train_images': train_images_dir,
        'train_labels': train_labels_dir,
        'test_images': test_images_dir,
        'test_labels': test_labels_dir,
        'train_patients': train_patients,
        'test_patients': test_patients
    }

Датасет

In [96]:
class SegmentationDataset(Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform

        self.image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.png')])
        self.label_files = sorted([f for f in os.listdir(labels_dir) if f.endswith('.png')])
    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = os.path.join(self.images_dir, self.image_files[idx])
        label_path = os.path.join(self.labels_dir, self.label_files[idx])
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        label = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)

        if self.transform:
            transformed = self.transform(image=image, mask=label)
            image = transformed['image']
            label = transformed['mask']
        label = (label > 0).long()

        return image, label

Модель Unet

In [97]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class UNet(nn.Module):
    def contracting_block(self, in_channels, out_channels, kernel_size=3):
        block = torch.nn.Sequential(
            torch.nn.Conv2d(kernel_size=kernel_size, in_channels=in_channels, out_channels=out_channels, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(out_channels),
            torch.nn.Conv2d(kernel_size=kernel_size, in_channels=out_channels, out_channels=out_channels, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(out_channels),
        )
        return block

    def expansive_block(self, in_channels, mid_channel, out_channels, kernel_size=3):
        block = torch.nn.Sequential(
            torch.nn.Conv2d(kernel_size=kernel_size, in_channels=in_channels, out_channels=mid_channel, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(mid_channel),
            torch.nn.Conv2d(kernel_size=kernel_size, in_channels=mid_channel, out_channels=mid_channel, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(mid_channel),
            torch.nn.ConvTranspose2d(in_channels=mid_channel, out_channels=out_channels, kernel_size=2, stride=2)
        )
        return block

    def final_block(self, in_channels, mid_channel, out_channels, kernel_size=3):
        block = torch.nn.Sequential(
            torch.nn.Conv2d(kernel_size=kernel_size, in_channels=in_channels, out_channels=mid_channel, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(mid_channel),
            torch.nn.Conv2d(kernel_size=kernel_size, in_channels=mid_channel, out_channels=mid_channel, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(mid_channel),
            torch.nn.Conv2d(kernel_size=kernel_size, in_channels=mid_channel, out_channels=out_channels, padding=1),
        )
        return block

    def __init__(self, in_channel, out_channel):
        super(UNet, self).__init__()
        self.conv_encode1 = self.contracting_block(in_channels=in_channel, out_channels=64)
        self.conv_maxpool1 = torch.nn.MaxPool2d(kernel_size=2)
        self.conv_encode2 = self.contracting_block(64, 128)
        self.conv_maxpool2 = torch.nn.MaxPool2d(kernel_size=2)
        self.conv_encode3 = self.contracting_block(128, 256)
        self.conv_maxpool3 = torch.nn.MaxPool2d(kernel_size=2)

        self.bottleneck = torch.nn.Sequential(
            torch.nn.Conv2d(kernel_size=3, in_channels=256, out_channels=512, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(512),
            torch.nn.Conv2d(kernel_size=3, in_channels=512, out_channels=512, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(512),
            torch.nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=2, stride=2)
        )

        self.conv_decode3 = self.expansive_block(512, 256, 128)
        self.conv_decode2 = self.expansive_block(256, 128, 64)
        self.final_layer = self.final_block(128, 64, out_channel)

    def crop_and_concat(self, upsampled, bypass):
        _, _, h_up, w_up = upsampled.size()
        _, _, h_by, w_by = bypass.size()

        crop_h = (h_by - h_up) // 2
        crop_w = (w_by - w_up) // 2

        bypass_cropped = bypass[:, :,
                              crop_h:crop_h + h_up,
                              crop_w:crop_w + w_up]

        return torch.cat((upsampled, bypass_cropped), 1)

    def forward(self, x):
        encode_block1 = self.conv_encode1(x)
        encode_pool1 = self.conv_maxpool1(encode_block1)

        encode_block2 = self.conv_encode2(encode_pool1)
        encode_pool2 = self.conv_maxpool2(encode_block2)

        encode_block3 = self.conv_encode3(encode_pool2)
        encode_pool3 = self.conv_maxpool3(encode_block3)

        bottleneck1 = self.bottleneck(encode_pool3)

        decode_block3 = self.crop_and_concat(bottleneck1, encode_block3)
        cat_layer2 = self.conv_decode3(decode_block3)

        decode_block2 = self.crop_and_concat(cat_layer2, encode_block2)
        cat_layer1 = self.conv_decode2(decode_block2)

        decode_block1 = self.crop_and_concat(cat_layer1, encode_block1)
        final_layer = self.final_layer(decode_block1)

        return final_layer

Аугментация и предподготовка

In [98]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

def transformers():
    cfg = load_hydra_config()
    train_transform = A.Compose([
        A.Resize(height=cfg.transforms.image_size, width=cfg.transforms.image_size),

        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.3),
        A.RandomRotate90(p=0.5),
        A.Affine(
            translate_percent={'x': (-0.05, 0.05), 'y': (-0.05, 0.05)},
            scale=(0.9, 1.1),
            rotate=(-10, 10),
            p=0.5
        ),
        A.RandomBrightnessContrast(
            brightness_limit=0.1,
            contrast_limit=0.1,
            p=0.3
        ),
        A.GaussNoise(p=0.2),

        A.Normalize(mean=[0.5], std=[0.5]),
        ToTensorV2(),
    ])

    val_transform = A.Compose([
        A.Resize(height=cfg.transforms.image_size, width=cfg.transforms.image_size),
        A.Normalize(mean=[0.5], std=[0.5]),
        ToTensorV2(),
    ])

    return train_transform, val_transform

In [99]:
!pip install torchmetrics
from torchmetrics.segmentation import DiceScore
from torchmetrics import JaccardIndex
from torchmetrics import Precision, Recall


In [100]:
def train_model():
    cfg = load_hydra_config()
    task = Task.init(
        project_name="MedicalSegmentation",
        task_name=f"UNet_20epochs",
        auto_connect_frameworks={'pytorch': True, 'hydra': True}
    )
    cfg_dict = OmegaConf.to_container(cfg, resolve=True)
    task.connect_configuration(cfg_dict)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    paths = create_train_test_split_by_patient()
    train_transform, val_transform = transformers()
    train_dataset = SegmentationDataset(
        images_dir=paths['train_images'],
        labels_dir=paths['train_labels'],
        transform=train_transform
    )

    test_dataset = SegmentationDataset(
        images_dir=paths['test_images'],
        labels_dir=paths['test_labels'],
        transform=val_transform
    )
    train_loader = DataLoader(
        train_dataset,
        batch_size=cfg.training.batch_size,
        shuffle=True,
        num_workers=cfg.data.num_workers
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=cfg.training.batch_size,
        shuffle=False,
        num_workers=cfg.data.num_workers
    )

    model = UNet(cfg.model.in_channel, cfg.model.out_channel).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=cfg.training.learning_rate,
        weight_decay=cfg.training.weight_decay
    )
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min')
    dice_metric = DiceScore(num_classes=2, average='macro', input_format='index').to(device)
    iou_metric = JaccardIndex(num_classes=2, task='multiclass', average='macro').to(device)
    precision_metric = Precision(num_classes=2, task='multiclass',average='macro').to(device)
    recall_metric = Recall(num_classes=2, task='multiclass',average='macro').to(device)

    train_losses = []
    val_dice_scores = []
    val_iou_scores = []
    val_pres_scores = []
    val_rec_scores = []
    best_dice = 0
    best_epoch = 0
    for epoch in range(cfg.training.epochs):
        model.train()
        epoch_train_loss = 0
        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch}/{cfg.training.epochs} [Train]')

        for idx, (images, labels) in enumerate(train_pbar):
            images, labels = images.to(device), labels.to(device)
            labels = labels.long()
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            epoch_train_loss += loss.item()

            train_pbar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Avg Loss': f'{epoch_train_loss/(idx+1):.4f}'
            })

        train_pbar.close()
        avg_train_loss = epoch_train_loss / len(train_loader)

        model.eval()
        epoch_val_dice = 0
        val_batches = 0
        val_pbar = tqdm(test_loader, desc=f'Epoch {epoch+1}/{cfg.training.epochs} [Val]')


        dice_metric.reset()
        iou_metric.reset()
        precision_metric.reset()
        recall_metric.reset()

        with torch.no_grad():
            for idx, (images, masks) in enumerate(val_pbar):
                images, masks = images.to(device), masks.to(device)
                masks = masks.long()
                outputs = model(images)
                probs = torch.softmax(outputs, dim=1)
                preds = torch.argmax(probs, dim=1)

                dice_metric.update(preds, masks)
                iou_metric.update(preds, masks)
                precision_metric.update(preds, masks)
                recall_metric.update(preds, masks)

                val_batches += 1
        val_pbar.close()
        avg_val_dice = dice_metric.compute()
        avg_val_iou = iou_metric.compute()
        avg_val_pres = precision_metric.compute()
        avg_val_rec = recall_metric.compute()
        if avg_val_dice > best_dice:
            best_dice = avg_val_dice.item()
            best_epoch = epoch
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'dice_score': best_dice,
                'iou_score': avg_val_iou.item(),
                'loss': avg_train_loss,
            }, 'best_model.pth')

        logger = Logger.current_logger()
        logger.report_scalar("Loss", "Train", avg_train_loss, epoch)
        logger.report_scalar("Metrics", "Dice", avg_val_dice.item(), epoch)
        logger.report_scalar("Metrics", "Iou", avg_val_iou.item(), epoch)
        logger.report_scalar("Metrics", "Precision", avg_val_pres.item(), epoch)
        logger.report_scalar("Metrics", "Recall", avg_val_rec.item(), epoch)

        train_losses.append(avg_train_loss)
        val_dice_scores.append(avg_val_dice.item())
        val_iou_scores.append(avg_val_iou.item())
        val_pres_scores.append(avg_val_pres.item())
        val_rec_scores.append(avg_val_rec.item())

        scheduler.step(avg_train_loss)

        print(f'Epoch {epoch:03d}:')
        print(f'  Loss: {avg_train_loss:.4f}')
        print(f'  Dice: {avg_val_dice.item():.4f}')
        print(f'  Iou: {avg_val_iou.item():.4f}')
        print(f'  Precision: {avg_val_pres.item():.4f}')
        print(f'  Recall: {avg_val_rec.item():.4f}')
        print(f'  LR:   {optimizer.param_groups[0]["lr"]:.2e}')
        print(f'Best Dice {best_dice:.4f} Epoch {best_epoch}')

In [101]:
if __name__ == "__main__":
    results = train_model()

Epoch 1/40 [Val]: 100%|██████████| 133/133 [00:37<00:00,  3.58it/s]


Epoch 000:
  Loss: 0.2387
  Dice: 0.5935
  Iou: 0.5830
  Precision: 0.8335
  Recall: 0.5933
  LR:   2.00e-04
Best Dice 0.5935 Epoch 0


Epoch 2/40 [Val]: 100%|██████████| 133/133 [00:38<00:00,  3.49it/s]


Epoch 001:
  Loss: 0.0240
  Dice: 0.8037
  Iou: 0.8263
  Precision: 0.9490
  Recall: 0.8530
  LR:   2.00e-04
Best Dice 0.8037 Epoch 1


Epoch 3/40 [Val]: 100%|██████████| 133/133 [00:38<00:00,  3.45it/s]


...

Epoch 037:
  Loss: 0.0035
  Dice: 0.9659
  Iou: 0.9673
  Precision: 0.9860
  Recall: 0.9802
  LR:   2.00e-04
Best Dice 0.9659 Epoch 37


Epoch 39/40 [Val]: 100%|██████████| 133/133 [00:37<00:00,  3.51it/s]


Epoch 038:
  Loss: 0.0036
  Dice: 0.9653
  Iou: 0.9654
  Precision: 0.9842
  Recall: 0.9799
  LR:   2.00e-04
Best Dice 0.9659 Epoch 37


Epoch 40/40 [Val]: 100%|██████████| 133/133 [00:37<00:00,  3.55it/s]

Epoch 039:
  Loss: 0.0035
  Dice: 0.9659
  Iou: 0.9651
  Precision: 0.9830
  Recall: 0.9809
  LR:   2.00e-04
Best Dice 0.9659 Epoch 37
